# Azure完成示例

> 注意：有一个更新版本的openai库可用。请参阅 https://github.com/openai/openai-python/discussions/742

本示例将涵盖使用Azure OpenAI服务的完成功能。


## 设置


首先，我们安装必要的依赖项。


In [ ]:
! pip install "openai>=0.28.1,<1.0.0"


为了使以下部分正常工作，我们首先需要设置一些内容。让我们从 `api_base` 和 `api_version` 开始。要找到你的 `api_base`，请转到 https://portal.azure.com，在资源下找到你的资源，然后在 "资源管理" -> "密钥和端点" 下查找 "端点" 值。


In [ ]:
import os
import openai


In [ ]:
openai.api_version = '2023-05-15'
openai.api_base = '' # 请在此处添加您的端点


接下来我们需要设置`api_type`和`api_key`。我们可以从门户获取密钥，也可以通过Microsoft Active Directory身份验证获取密钥。根据这一点，`api_type`可以是`azure`或`azure_ad`。


### 设置：门户
首先让我们看看如何从门户获取密钥。转到 https://portal.azure.com，找到您的资源，然后在“资源管理” -> “密钥和端点”下查找一个“密钥”值。


In [ ]:
openai.api_type = 'azure'
openai.api_key = os.environ["OPENAI_API_KEY"]


> 注意：在这个例子中，我们通过在代码中设置变量来配置库以使用Azure API。对于开发，考虑设置环境变量而不是在代码中设置：

```
OPENAI_API_BASE
OPENAI_API_KEY
OPENAI_API_TYPE
OPENAI_API_VERSION
```


### (可选) 设置：Microsoft Active Directory身份验证
现在让我们看看如何通过Microsoft Active Directory身份验证获取密钥。如果您想使用Active Directory身份验证而不是从门户获取密钥，请取消下面代码的注释。


In [ ]:
# 从 azure.identity 导入 DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token


令牌在一段时间内有效，之后将过期。为了确保每个请求都发送一个有效的令牌，您可以通过连接到requests.auth来刷新一个即将过期的令牌：


In [ ]:
import typing
import time
import requests
if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

session = requests.Session()
session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

openai.requestssession = session


## 部署
在本节中，我们将使用`gpt-3.5-turbo-instruct`模型创建一个部署，然后我们可以使用它来生成完成结果。


### 部署：手动创建
通过在门户中的“资源管理”->“模型部署”下找到您的资源，创建一个新的部署。选择`gpt-3.5-turbo-instruct`作为模型。


In [ ]:
deployment_id = '' # 在此处填写从门户获取的部署ID


### 完成
现在让我们向部署发送一个样本完成。


In [ ]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id=deployment_id,
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")
